In [2]:
#!pip install lightgbm

In [3]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [7]:
independent=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [8]:
dependent=dataset[['charges']]
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [9]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent, dependent, test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler 
sc=StandardScaler()
X_train=sc.fit_transform(X_train) 
X_test=sc.transform(X_test)

In [12]:
param_grid = {
    'boosting_type': ['gbdt'], 
    'num_leaves': [31], 
    'max_depth': [-1], 
    'learning_rate': [0.1], 
    'n_estimators': [100], 
    'subsample_for_bin': [200000], 
    'objective': [None],  # Assuming you meant to pass None, not 'None'
    'class_weight': [None], 
    'min_split_gain': [0.0], 
    'min_child_weight': [0.001], 
    'min_child_samples': [20], 
    'subsample': [1.0], 
    'subsample_freq': [0],
    'force_row_wise': [True],  # Use Python boolean True instead of "true"
    'colsample_bytree': [1.0], 
    'reg_alpha': [0.0], 
    'reg_lambda': [0.0], 
    'random_state': [None], 
    'n_jobs': [None],  # Use None without quotes
    'importance_type': ['split']
}

grid = GridSearchCV(LGBMRegressor(), param_grid, refit = True, verbose = 3,n_jobs=-1) 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 5
[LightGBM] [Info] Start training from score 13232.916456


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'boosting_type': ['gbdt'], 'class_weight': [None],
                         'colsample_bytree': [1.0], 'force_row_wise': [True],
                         'importance_type': ['split'], 'learning_rate': [0.1],
                         'max_depth': [-1], 'min_child_samples': [20],
                         'min_child_weight': [0.001], 'min_split_gain': [0.0],
                         'n_estimators': [100], 'n_jobs': [None],
                         'num_leaves': [31], 'objective': [None],
                         'random_state': [None], 'reg_alpha': [0.0],
                         'reg_lambda': [0.0], 'subsample': [1.0],
                         'subsample_for_bin': [200000], 'subsample_freq': [0]},
             verbose=3)

In [13]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test)

# print classification report
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)
print("The R_score value for best parameter {}:".format(grid.best_params_),r_score)

The R_score value for best parameter {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'force_row_wise': True, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}: 0.8699321391117371


In [ ]:
table=pd.DataFrame.from_dict(re)

In [ ]:
table

In [ ]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

In [ ]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))